Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Register Model and deploy as Webservice

This example shows how to deploy a Webservice in step-by-step fashion:

 1. Register Model
 2. Deploy Model as Webservice

## Prerequisites
1) If you are using an Azure Machine Learning Notebook VM, you are all set. 

2) Run through [1_Bert_StackOverflow_Training](1_Bert_StackOverflow_Training.ipynb) Notebook first to register your model

In [7]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.62


## Initialize Workspace

Initialize a workspace object from persisted configuration.

In [8]:
from azureml.core import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: abe-gpu-ws
Azure region: westus2
Subscription id: 15ae9cb6-95c1-483d-a0e3-b1a1a3b06324
Resource group: osomorog


### Retrieve the Model

You can retrieve your BERT Model created in the first notebook. [1_Bert_StackOverflow_Training](1_Bert_StackOverflow_Training.ipynb) Notebook first to register your model

In [9]:
from azureml.core import Model
model = ws.models['bert-stackoverflow']
model

Model(workspace=Workspace.create(name='abe-gpu-ws', subscription_id='15ae9cb6-95c1-483d-a0e3-b1a1a3b06324', resource_group='osomorog'), name=bert-stackoverflow, id=bert-stackoverflow:1, version=1, tags={}, properties={})

#### Download bert-stackoverflow source code into local repository and install submodules

In [5]:
print('Downloading repo...')
project_folder = './bert-stackoverflow'
os.makedirs(project_folder, exist_ok=True)
os.system('git clone https://github.com/AbeOmor/bert-stackoverflow/tree/patch-1 && cd bert-stackoverflow && git submodule update --init --recursive && cd ..')
print('Done.')

Done.


## Deploy as web service

Once you've tested the model and are satisfied with the results, deploy the model as a web service hosted in ACI. 

To build the correct environment for ACI, provide the following:
* A scoring script to show how to use the model
* An environment file to show what packages need to be installed
* A configuration file to build the ACI
* The model you trained before

### Create Environment

You can now create and/or use an Environment object when deploying a Webservice. The Environment can have been previously registered with your Workspace, or it will be registered with it as a part of the Webservice deployment. Only Environments that were created using azureml-defaults version 1.0.48 or later will work with this new handling however.

In [10]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies.create(conda_packages=['numpy','pandas'],
                                 pip_packages=['numpy','pandas','inference-schema[numpy-support]','azureml-defaults','tensorflow==1.13.2'])

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

Review the content of the `myenv.yml` file.

In [11]:
with open("myenv.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - numpy
  - pandas
  - inference-schema[numpy-support]
  - azureml-defaults==1.0.62.*
  - tensorflow==1.13.2
- numpy
- pandas
channels:
- conda-forge



### Create configuration file

Create a deployment configuration file and specify the number of CPUs and gigabyte of RAM needed for your ACI container. While it depends on your model, the default of 1 core and 1 gigabyte of RAM is usually sufficient for many models. If you feel you need more later, you would have to recreate the image and redeploy the service.

In [12]:
from azureml.core.webservice import AciWebservice, Webservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"model": "BERT",  "method" : "tensorflow"}, 
                                               description='Predict StackoverFlow tags with BERT')

## Create Inference Configuration

There is now support for a source directory, you can upload an entire folder from your local machine as dependencies for the Webservice.
Note: in that case, your entry_script, conda_file, and extra_docker_file_steps paths are relative paths to the source_directory path.

Sample code for using a source directory:

```python
inference_config = InferenceConfig(source_directory="C:/abc",
                                   runtime= "python", 
                                   entry_script="x/y/score.py",
                                   conda_file="env/myenv.yml", 
                                   extra_docker_file_steps="helloworld.txt")
```

 - source_directory = holds source path as string, this entire folder gets added in image so its really easy to access any files within this folder or subfolder
 - runtime = Which runtime to use for the image. Current supported runtimes are 'spark-py' and 'python
 - entry_script = contains logic specific to initializing your model and running predictions
 - conda_file = manages conda and python package dependencies.
 - extra_docker_file_steps = optional: any extra steps you want to inject into docker file

In [13]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(source_directory="bert-stackoverflow",
                                   runtime= "python", 
                                   entry_script="score.py",
                                   conda_file="../myenv.yml")

### Deploy in ACI
Estimated time to complete: **about 5-10 minutes**

Configure the image and deploy. The following code goes through these steps:

* Build an image using:
   * The scoring file (`score.py`)
   * The environment file (`myenv.yml`)
   * The model file
* Register that image under the workspace. 
* Send the image to the ACI container.
* Start up a container in ACI using the image.
* Get the web service HTTP endpoint.

In [30]:
%%time
from azureml.core.webservice import Webservice
from azureml.exceptions import WebserviceException

aci_service_name = 'bert-stackoverflow-aciservice'

try:
    # if you want to get existing service below is the command
    # since aci name needs to be unique in subscription deleting existing aci if any
    # we use aci_service_name to create azure aci
    service = Webservice(ws, name=aci_service_name)
    if service:
        service.delete()
except WebserviceException as e:
    print()

service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)

service.wait_for_deployment(True)
print(service.state)

Running..........................
SucceededACI service creation operation finished, operation "Succeeded"
Healthy
CPU times: user 569 ms, sys: 98.2 ms, total: 667 ms
Wall time: 2min 54s


#### Test web service

In [38]:
%%time
import json
test_sample = json.dumps({
    'data': [
        {
        'id': 123,
        'text': 'I need help with importing a module with tensorflow 2.0'
        }
    ]
})

#test_sample_encoded = bytes(test_sample, encoding='utf8')
prediction = service.run(input_data=test_sample)
print(prediction)

[
  {
    "id": "123",
    "text": "I need help with importing a module with tensorflow 2.0",
    "probabilities": {
      "c#": "0.012589216",
      ".net": "0.015036851",
      "java": "0.038416177",
      "asp.net": "0.016928166",
      "c++": "0.046544015",
      "javascript": "0.068582",
      "php": "0.01960367",
      "python": "0.29779866",
      "sql": "0.009429228",
      "sql-server": "0.009171811"
    }
  }
]
CPU times: user 3.92 ms, sys: 0 ns, total: 3.92 ms
Wall time: 1.3 s


### View ACI Logs

In [32]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(service.get_logs())

('/bin/bash: '
 '/azureml-envs/azureml_617e8a56e351dc1398e8e025d4c910d2/lib/libtinfo.so.5: no '
 'version information available (required by /bin/bash)\n'
 '/bin/bash: '
 '/azureml-envs/azureml_617e8a56e351dc1398e8e025d4c910d2/lib/libtinfo.so.5: no '
 'version information available (required by /bin/bash)\n'
 '/bin/bash: '
 '/azureml-envs/azureml_617e8a56e351dc1398e8e025d4c910d2/lib/libtinfo.so.5: no '
 'version information available (required by /bin/bash)\n'
 '/bin/bash: '
 '/azureml-envs/azureml_617e8a56e351dc1398e8e025d4c910d2/lib/libtinfo.so.5: no '
 'version information available (required by /bin/bash)\n'
 'bash: '
 '/azureml-envs/azureml_617e8a56e351dc1398e8e025d4c910d2/lib/libtinfo.so.5: no '
 'version information available (required by bash)\n'
 '/usr/sbin/nginx: '
 '/azureml-envs/azureml_617e8a56e351dc1398e8e025d4c910d2/lib/libcrypto.so.1.0.0: '
 'no version information available (required by /usr/sbin/nginx)\n'
 '/usr/sbin/nginx: '
 '/azureml-envs/azureml_617e8a56e351dc1398

#### Delete ACI to clean up

In [30]:
service.delete()